# Training of College helper Chatbot
### Tridib Dalui

In [1]:
import random
import json
import pickle
import numpy as np

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

In [4]:
# load the intents

with open('intents.json','r') as f:
   intents= json.load(f)

Now we're goint to lemmatize each word
=> works,worked,working==== work

In [5]:
lt=WordNetLemmatizer()

In [6]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)#tokenize=> "helo me god"==["hello","me","god"]
        words.extend(word_list) #appending each word in word list
        documents.append((word_list,intent['tag'])) # storing word and their tags
        if intent['tag'] not in classes:
            classes.append(intent['tag'])#appending the tags in classes
#now lemmatizing the words        
words=[lt.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
classes=sorted(set(classes))

In [8]:
print(words)

["'s", '(', ')', 'AC', 'AI/ML', 'AI/Ml', 'About', 'Ac', 'Are', 'Bye', 'Cafetaria', 'Can', 'Chemical', 'Civil', 'College', 'Computer', 'Contact', 'Do', 'Does', 'Engineering', 'Events', 'Food', 'Give', 'Good', 'Goodbye', 'HOD', 'Have', 'Hello', 'Hi', 'Holiday', 'How', 'I', 'IT', 'Information', 'Is', 'Leaving', 'Library', 'ME', 'Mechanical', 'See', 'Semester', 'Sports', 'Syllabus', 'Technology', 'Tell', 'Thank', 'Thanks', 'UNI', 'What', 'Whats', 'When', 'Where', 'Which', 'Who', 'Will', 'You', 'a', 'about', 'active', 'activity', 'address', 'admision', 'admission', 'against', 'allotment', 'am', 'an', 'and', 'antiragging', 'any', 'anyone', 'are', 'as', 'asshole', 'at', 'attend', 'automobile', 'available', 'average', 'be', 'between', 'big', 'bitch', 'book', 'boy', 'branch', 'bring', 'building', 'by', 'bye', 'cafetaria', 'call', 'called', 'campus', 'can', 'canteen', 'capacity', 'case', 'casuals', 'ce', 'chatting', 'chemical', 'civil', 'code', 'college', 'come', 'committee', 'comp', 'company', 

In [9]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

now we will use bag of words technique to convert the words to neumerical values as neural network uses neumerical values

In [10]:
training=[]
output_empty=[0]*len(classes)

for document in documents:
    bag=[] #crearing an empty bag
    word_patterns=document[0]; #as the word is the first element of document
    word_patterns=[lt.lemmatize(word.lower()) for word in word_patterns] #lemmatizing the lowercase words for training list
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1 #mark the selected class of document as 1
    training.append([bag,output_row])

In [11]:
#shuffel the data randomly for training

random.shuffle(training)
training=np.array(training)

<ipython-input-11-362043d35415>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [12]:
#input & output data

train_x=list(training[:,0])
train_y=list(training[:,1])

##### Model Building

In [13]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               39168     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 38)                2470      
                                                                 
Total params: 49,894
Trainable params: 49,894
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [16]:
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=10,verbose=1)

Epoch 1/200
41/41 [==============================] - 1s 2ms/step - loss: 3.6106 - accuracy: 0.0517
Epoch 2/200
41/41 [==============================] - 0s 2ms/step - loss: 3.5313 - accuracy: 0.0837
Epoch 3/200
41/41 [==============================] - 0s 2ms/step - loss: 3.4334 - accuracy: 0.1034
Epoch 4/200
41/41 [==============================] - 0s 2ms/step - loss: 3.2732 - accuracy: 0.1552
Epoch 5/200
41/41 [==============================] - 0s 2ms/step - loss: 3.1692 - accuracy: 0.1773
Epoch 6/200
41/41 [==============================] - 0s 2ms/step - loss: 2.9823 - accuracy: 0.2315
Epoch 7/200
41/41 [==============================] - 0s 2ms/step - loss: 2.7887 - accuracy: 0.2980
Epoch 8/200
41/41 [==============================] - 0s 2ms/step - loss: 2.6981 - accuracy: 0.2931
Epoch 9/200
41/41 [==============================] - 0s 2ms/step - loss: 2.5190 - accuracy: 0.3473
Epoch 10/200
41/41 [==============================] - 0s 2ms/step - loss: 2.4549 - accuracy: 0.3571
Epoch 11/

41/41 [==============================] - 0s 2ms/step - loss: 0.2537 - accuracy: 0.9187
Epoch 84/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2282 - accuracy: 0.9532
Epoch 85/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2740 - accuracy: 0.9089
Epoch 86/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2601 - accuracy: 0.9360
Epoch 87/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2434 - accuracy: 0.9236
Epoch 88/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2360 - accuracy: 0.9187
Epoch 89/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2362 - accuracy: 0.9286
Epoch 90/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2331 - accuracy: 0.9335
Epoch 91/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2504 - accuracy: 0.9335
Epoch 92/200
41/41 [==============================] - 0s 2ms/step - loss: 0.2223 - accuracy: 0.9483
Epoch 93/200


41/41 [==============================] - 0s 1ms/step - loss: 0.1677 - accuracy: 0.9557
Epoch 165/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1395 - accuracy: 0.9409
Epoch 166/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1593 - accuracy: 0.9532
Epoch 167/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1485 - accuracy: 0.9581
Epoch 168/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1514 - accuracy: 0.9507
Epoch 169/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1597 - accuracy: 0.9458
Epoch 170/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1642 - accuracy: 0.9507
Epoch 171/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1253 - accuracy: 0.9655
Epoch 172/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1407 - accuracy: 0.9606
Epoch 173/200
41/41 [==============================] - 0s 1ms/step - loss: 0.1364 - accuracy: 0.9606
Epoc

In [17]:
model.save('CollegeBuddy_model.h5',hist)